In [ ]:
# config.py
import os
from dotenv import load_dotenv

load_dotenv()

class Config:
    # HyperCLOVA X API 설정
    NCP_CLOVASTUDIO_API_KEY = os.getenv('NCP_CLOVASTUDIO_API_KEY')
    
    # 다른 API 키들
    ALPHA_VANTAGE_KEY = os.getenv('ALPHA_VANTAGE_KEY')
    FINNHUB_TOKEN = os.getenv('FINNHUB_TOKEN')
    
    # 기본 설정
    DEFAULT_TICKERS = ['005930.KS', '000660.KS', '035420.KS', '051910.KS', '207940.KS']  # 한국 주식
    CACHE_DURATION = 300
    MAX_NEWS_ARTICLES = 20


In [6]:
# agents/sentiment_analyzer.py
import asyncio
from typing import List, Dict, Any
import logging
from hyperclova_client import HyperCLOVAXClient

logger = logging.getLogger(__name__)

class SentimentAnalyzerAgent:
    def __init__(self):
        self.clova_client = HyperCLOVAXClient()
        
    async def analyze_news_sentiment(self, news_articles: List[Dict[str, str]]) -> Dict[str, Any]:
        """HyperCLOVA X를 사용한 뉴스 감정 분석"""
        sentiment_results = {}
        
        try:
            for ticker in set(article['ticker'] for article in news_articles):
                ticker_articles = [article for article in news_articles if article['ticker'] == ticker]
                
                if not ticker_articles:
                    continue
                
                # 뉴스 텍스트 결합
                combined_text = "\n".join([
                    f"제목: {article['title']}\n요약: {article['summary']}" 
                    for article in ticker_articles[:5]  # 최대 5개 기사
                ])
                
                # HyperCLOVA X로 감정 분석
                sentiment_analysis = await self._analyze_sentiment_with_clova(combined_text, ticker)
                
                sentiment_results[ticker] = {
                    'sentiment_score': sentiment_analysis['score'],
                    'confidence': sentiment_analysis['confidence'],
                    'article_count': len(ticker_articles),
                    'analysis_summary': sentiment_analysis['summary'],
                    'method': 'hyperclova_x'
                }
                
                # API 호출 제한을 위한 지연
                await asyncio.sleep(0.5)
                
        except Exception as e:
            logger.error(f"감정 분석 오류: {e}")
            
        return sentiment_results
    
    async def _analyze_sentiment_with_clova(self, text: str, ticker: str) -> Dict[str, Any]:
        """HyperCLOVA X를 사용한 개별 감정 분석"""
        
        prompt = f"""
다음은 {ticker} 종목과 관련된 뉴스 기사들입니다. 이 기사들의 전반적인 감정을 분석해주세요.

뉴스 내용:
{text}

다음 형식으로 응답해주세요:
1. 감정 점수: -1.0(매우 부정) ~ +1.0(매우 긍정) 사이의 숫자
2. 신뢰도: 0.0 ~ 1.0 사이의 숫자
3. 분석 요약: 2-3문장으로 감정 분석 결과 설명

예시:
감정 점수: 0.3
신뢰도: 0.8
분석 요약: 전반적으로 긍정적인 뉴스가 많으며, 실적 개선과 신제품 출시 소식이 주요 호재로 작용하고 있습니다.
"""
        
        messages = [
            {"role": "system", "content": "당신은 한국 주식시장 전문 감정 분석가입니다. 뉴스 기사의 감정을 정확하게 분석해주세요."},
            {"role": "user", "content": prompt}
        ]
        
        try:
            response = await self.clova_client.generate_response(messages, max_tokens=400)
            
            # 응답 파싱
            sentiment_score = self._extract_sentiment_score(response)
            confidence = self._extract_confidence(response)
            summary = self._extract_summary(response)
            
            return {
                'score': sentiment_score,
                'confidence': confidence,
                'summary': summary
            }
            
        except Exception as e:
            logger.error(f"HyperCLOVA X 감정 분석 오류: {e}")
            return {
                'score': 0.0,
                'confidence': 0.5,
                'summary': '분석 중 오류가 발생했습니다.'
            }
    
    def _extract_sentiment_score(self, response: str) -> float:
        """응답에서 감정 점수 추출"""
        try:
            lines = response.split('\n')
            for line in lines:
                if '감정 점수' in line or '점수' in line:
                    # 숫자 추출
                    import re
                    numbers = re.findall(r'-?\d+\.?\d*', line)
                    if numbers:
                        score = float(numbers[0])
                        return max(-1.0, min(1.0, score))  # -1.0 ~ 1.0 범위로 제한
            return 0.0
        except:
            return 0.0
    
    def _extract_confidence(self, response: str) -> float:
        """응답에서 신뢰도 추출"""
        try:
            lines = response.split('\n')
            for line in lines:
                if '신뢰도' in line:
                    import re
                    numbers = re.findall(r'\d+\.?\d*', line)
                    if numbers:
                        confidence = float(numbers[0])
                        return max(0.0, min(1.0, confidence))  # 0.0 ~ 1.0 범위로 제한
            return 0.7  # 기본값
        except:
            return 0.7
    
    def _extract_summary(self, response: str) -> str:
        """응답에서 분석 요약 추출"""
        try:
            lines = response.split('\n')
            for i, line in enumerate(lines):
                if '분석 요약' in line or '요약' in line:
                    if ':' in line:
                        return line.split(':', 1)[1].strip()
                    elif i + 1 < len(lines):
                        return lines[i + 1].strip()
            return response.split('\n')[-1].strip()  # 마지막 줄 반환
        except:
            return "분석 요약을 추출할 수 없습니다."
